In [1]:
import requests
import pandas as pd
from functools import *
import numpy as np
import nltk
%load_ext autoreload
%autoreload 2
import recip as rr
import dill
import submatcher as sm
import hashlib
import re

In [2]:
inp = ["flour","butter","milk", "egg", "cheese"]

In [4]:
allr = rr.GetRecipies(inp, recnum = 9)


Recipe cache found
recipe Fluffy Mozzarella Biscuits cache found
recipe Berry sweet empanadas cache found
recipe Ocean-themed Vanilla Cupcakes for Broadway’s "Anything Goes cache found
recipe Cabrales Cheese Souffles with Endive and Asian Pear Salad cache found
recipe Parmesan Puffs cache found
recipe Cheesy Popovers cache found
recipe Fancy French Cheese Puff Fake-out cache found
recipe Parmesan Herb Popovers Recipe cache found
recipe Cheese Puffs cache found


In [7]:
allr.to_json()

{'Fluffy Mozzarella Biscuits': {'attrs': {'vegetarian': True,
   'vegan': False,
   'glutenFree': False,
   'dairyFree': False},
  'c': True,
  'cuisines': [],
  'id': 559259,
  'imglink': 'https://spoonacular.com/recipeImages/559259-312x231.jpg',
  'missedCount': 2,
  'missedIngs': ['2 tbsp baking powder', '2 cups shredded mozzarella cheese'],
  'name': 'Fluffy Mozzarella Biscuits',
  'price': 34.01,
  'serving': 18,
  'subIngs': {'baking powder': {'baking powder': ['baking soda plus cream of tartar ',
     ' baking soda plus buttermilk (decrease liquid in recipe by ']},
   'shredded mozzarella cheese': {'cheddar cheese': ['shredded Colby cheddar ',
     'shredded Monterey Jack cheese'],
    'cottage cheese': ["armer's cheese ", 'ricotta cheese'],
    'cream cheese': ['pureed cottage cheese ',
     'plain yogurt, strained overnight in a cheesecloth'],
    "farmer's cheese": ['dry cottage cheese ',
     'creamed cottage cheese, drained'],
    'parmesan cheese': ['grated Asiago cheese '

In [5]:
tbdf = sm.makesubdict("https://www.allrecipes.com/article/common-ingredient-substitutions/")

In [8]:
import json
with open('testout.json', 'w', encoding='utf-8') as f:
    json.dump(allr.to_json(), f)